In [ ]:
# ========================================
# SECTION 4: Multi-Step Pipeline
# ========================================
# In this section, you'll BUILD a real DSPy pipeline!
#
# A pipeline chains multiple steps together:
# 1. Analyze the code issue and generate reflection
# 2. Fix the code based on all available data
#
# We'll provide the signatures - you build the module!
# ========================================

In [ ]:
from utils.helper import get_api_key, validate_prediction
import dspy

In [ ]:
# Configure your values here
model_name = 'groq/llama-3.1-8b-instant'
api_key = get_api_key('GROQ_API_KEY')
api_endpoint = 'https://api.groq.com/openai/v1'

In [ ]:
llm = dspy.LM(
    model_name,
    api_key=api_key,
    api_base=api_endpoint,
)

# Set default LLM
dspy.settings.configure(lm=llm)

Let's define example data with a syntax error and unit tests

In [ ]:
snippet = """
def build_query(table, columns, condition):
  query = "SELECT " + ", ".join(columns) +
          " FROM " + table +
          " WHERE " condition
  return query
"""
traceback = "invalid syntax (<unknown>, line 2)"

unit_test = """
result = build_query("users", ["id", "name", "email"], "active = 1")
assert "SELECT id, name, email" in result
assert "FROM users" in result
assert "WHERE active = 1" in result
"""

First, let's define required signatures and custom module for fixing the issue.

In [ ]:
# ========================================
# PROVIDED: Signatures (use these as-is)
# ========================================

class AnalyzeSignature(dspy.Signature):
    """Explain the problem in the code"""
    snippet = dspy.InputField(description = "Code snippet")
    context = dspy.InputField(description = "Extra context about issue, like syntax error, etc.")
    summary = dspy.OutputField(description = "Issue details")
    confidence: float = dspy.OutputField(description = "Confidence about issue details")

class FixSignature(dspy.Signature):
    """Fix the code based on analysis"""
    snippet = dspy.InputField(description = "Code snippet")
    context = dspy.InputField(description = "Extra context about issue, like syntax error, etc.")
    analysis = dspy.InputField(description = "Analysis of the issue")
    fixed_code = dspy.OutputField(description = "Fixed code snippet")

In [ ]:
# ========================================
# TASK 1: Build the CodeFixer Module
# ========================================
# Your goal: Create a module that chains analysis → fix
#
# Steps:
# 1. In forward: Run analyze, then use its output for fix
# 2. Return the fixed code
#
# Hints:
# - Use dspy.ChainOfThought for analysis (gets reasoning)
# - Use dspy.Predict for fixing (direct fix)
# - The forward method gets 'snippet' and 'context' as inputs
# ========================================

class CodeFixer(dspy.Module):
    """Module to analyze and fix code issues"""

    def __init__(self):
        self.analyze = dspy.ChainOfThought(AnalyzeSignature)
        self.fix = dspy.Predict(FixSignature)

    # Forward method required by dspy.Module class
    def forward(self, snippet: str, context: str) -> str:
        # TODO: Implement the pipeline
        # 1. Call self.analyze with snippet and context
        # 2. Call self.fix with snippet, context, AND analysis results
        # 3. Return the fixed code

        # Hint: analysis_res = self.analyze(snippet=snippet, context=context)
        # Then access: analysis_res.summary

        analysis_res = self.analyze(snippet=snippet, context=context)
        fix_res = self.fix(snippet=snippet, context=context, analysis=analysis_res.summary)
        return fix_res.fixed_code

# Create your fixer
fixer = CodeFixer()

In [ ]:
# Run fixer here
fixed_snippet = fixer(snippet=snippet, context=traceback)
print("Fixed Snippet:")
print(fixed_snippet)
score, comment = validate_prediction(fixed_snippet, unit_test)
print(f"Score: {score}, Comment: {comment}")

In [ ]:
# ========================================
# TASK 2: Test Your Pipeline
# ========================================
# Run your CodeFixer and see if it works!
# ========================================

# YOUR CODE HERE:
# 1. Run: fixed_snippet = fixer(snippet=snippet, context=traceback)
# 2. Print the result
# 3. Validate it with the test

# fixed_snippet = fixer(???)
# print("Fixed Code:")
# print(fixed_snippet)

# Validate the fix
# score, comment = validate_prediction(fixed_snippet, unit_test)
# print(f"\nScore: {score}")
# print(f"Comment: {comment}")

In [ ]:
# ========================================
# Fix Iteratively (BONUS)
# ========================================
# Sometimes the first fix isn't perfect!
# Try running the fixer again on its own output
# ========================================
fixed_snippet = snippet
context = traceback
max_iterations = 3
for i in range(max_iterations):
    fixed_snippet = fixer(snippet=fixed_snippet, context=context)
    score, comment = validate_prediction(fixed_snippet, unit_test)
    print(f"Iteration {i+1}:\n\tScore = {score},\n\tComment: {comment}")
    if score == 1.0:
        print("✓ Fixed successfully!")
        print("Fixed Snippet:")
        print(fixed_snippet)
        break